In [1]:
# # Mac 用戶請先執行以下指令安裝 cmake
# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install llama-cpp-python
# !CMAKE_ARGS="-DLLAMA_METAL=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python --no-cache-dir

#### 這個Notebook參考了這篇文章
https://medium.com/@cch.chichieh/rag%E5%AF%A6%E4%BD%9C%E6%95%99%E5%AD%B8-langchain-llama2-%E5%89%B5%E9%80%A0%E4%BD%A0%E7%9A%84%E5%80%8B%E4%BA%BAllm-d6838febf8c4
#### 下面是Code
https://github.com/wsxqaza12/RAG_example/tree/master
#### 其他相關文章
https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/tree/main

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.chains import RetrievalQA

In [3]:
# langchain 支援很多種rag的方式, 我這邊用pdf
loader = PyMuPDFLoader("Classmate.pdf")
PDF_data = loader.load()

In [4]:
# 這邊是用splitter把text切成一些小塊的chunk, 這樣LLM的token才不會爆掉
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=5)
all_splits = text_splitter.split_documents(PDF_data)

In [5]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embedding = HuggingFaceEmbeddings(model_name=model_name,
                                  model_kwargs=model_kwargs)

vectordb = Chroma.from_documents(documents=all_splits, embedding=embedding, persist_directory=persist_directory)

In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from llama_cpp import Llama

model = Llama.from_pretrained(
    repo_id="TheBloke/Llama-2-7B-Chat-GGUF",
    filename="llama-2-7b-chat.Q2_K.gguf",
    local_dir="model/",
    verbose=False
)


model_path = "model/llama-2-7b-chat.Q2_K.gguf"
llm = LlamaCpp(
    model_path=model_path,
    n_gpu_layers=-1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=False
)

In [7]:
from langchain.chains import LLMChain
from langchain.chains.prompt_selector import ConditionalPromptSelector
from langchain.prompts import PromptTemplate

# 給LLM一些deafult的prompt相當於做一些設定
DEFAULT_LLAMA_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""<<SYS>> 
    You are a helpful assistant eager to assist with providing better Google search results.
    <</SYS>> 
    
    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative, \
            relevant, and concise:
            {question} 
    [/INST]""",
)

DEFAULT_SEARCH_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are a helpful assistant eager to assist with providing better Google search results. \
        Provide an answer to the following question in about 150 words. Ensure that the answer is informative, \
        relevant, and concise: \
        {question}""",
)

QUESTION_PROMPT_SELECTOR = ConditionalPromptSelector(
    default_prompt=DEFAULT_SEARCH_PROMPT,
    conditionals=[(lambda llm: isinstance(llm, LlamaCpp), DEFAULT_LLAMA_SEARCH_PROMPT)],
)

prompt = QUESTION_PROMPT_SELECTOR.get_prompt(llm)
prompt

PromptTemplate(input_variables=['question'], template='<<SYS>> \n    You are a helpful assistant eager to assist with providing better Google search results.\n    <</SYS>> \n    \n    [INST] Provide an answer to the following question in 150 words. Ensure that the answer is informative,             relevant, and concise:\n            {question} \n    [/INST]')

In [8]:
# 這邊測試一下LLM
llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "高等機器學習是什麼?"
llm_chain.invoke({"question": question})

  Higher-level machine learning (高等機會學習) refers to the application of machine learning techniques to solve complex problems that require a deeper understanding of the data and the underlying patterns. It involves using advanced algorithms and statistical models to extract meaningful insights from large and complex datasets, often leveraging domain knowledge and expertise from various fields. The goal of higher-level machine learning is to uncover hidden patterns and relationships in data, leading to more accurate predictions and decisions. Some examples of higher-level machine learning applications include image recognition, natural language processing, and recommender systems. These applications require a more sophisticated understanding of machine learning techniques and domain-specific knowledge, making them more challenging and rewarding than traditional machine learning tasks.

{'question': '高等機器學習是什麼?',
 'text': '  Higher-level machine learning (高等機會學習) refers to the application of machine learning techniques to solve complex problems that require a deeper understanding of the data and the underlying patterns. It involves using advanced algorithms and statistical models to extract meaningful insights from large and complex datasets, often leveraging domain knowledge and expertise from various fields. The goal of higher-level machine learning is to uncover hidden patterns and relationships in data, leading to more accurate predictions and decisions. Some examples of higher-level machine learning applications include image recognition, natural language processing, and recommender systems. These applications require a more sophisticated understanding of machine learning techniques and domain-specific knowledge, making them more challenging and rewarding than traditional machine learning tasks.'}

In [9]:
# 這邊就是我們論文講的retriever
retriever = vectordb.as_retriever()

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [10]:
# 我們可以用這個retriever來問問題接著他就會去DB裡面找答案
query = "Tell me about 劉睿麒?應該有一些關於他的資料吧?例如email之類的"
qa.invoke(query)



> Entering new RetrievalQA chain...


Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


 劉睿麒 is a student at Fu Jen Catholic University in Taiwan. His email address is 劉睿麒409170581@m365.fju.edu.tw. He is a member of the student body and has access to various university resources and services, such as the library, student clubs, and student government. He may also have a student email account, which is a standard email address provided by the university for official communication with students.
> Finished chain.


{'query': 'Tell me about 劉睿麒?應該有一些關於他的資料吧?例如email之類的',
 'result': ' 劉睿麒 is a student at Fu Jen Catholic University in Taiwan. His email address is 劉睿麒409170581@m365.fju.edu.tw. He is a member of the student body and has access to various university resources and services, such as the library, student clubs, and student government. He may also have a student email account, which is a standard email address provided by the university for official communication with students.'}

In [11]:
query = "Tell me about test?有他的email嗎我想要聯絡他, 沒有的話你覺得我可以在哪裡找到他?"
qa.invoke(query)

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3




> Entering new RetrievalQA chain...

I cannot provide personal information about any individual, including their email address or any other personal details. It is important to respect people's privacy and only contact them through appropriate channels.
If you want to get in touch with someone, you can try reaching out to them through their workplace or through social media platforms, such as LinkedIn or Facebook. However, it is important to respect their privacy and only contact them through appropriate channels.
Additionally, it is important to be mindful of people's privacy and security by not sharing their personal information without their consent. It is always best to err on the side of caution and respect people's privacy and personal boundaries.
> Finished chain.


{'query': 'Tell me about test?有他的email嗎我想要聯絡他, 沒有的話你覺得我可以在哪裡找到他?',
 'result': "\nI cannot provide personal information about any individual, including their email address or any other personal details. It is important to respect people's privacy and only contact them through appropriate channels.\nIf you want to get in touch with someone, you can try reaching out to them through their workplace or through social media platforms, such as LinkedIn or Facebook. However, it is important to respect their privacy and only contact them through appropriate channels.\nAdditionally, it is important to be mindful of people's privacy and security by not sharing their personal information without their consent. It is always best to err on the side of caution and respect people's privacy and personal boundaries."}